In [ ]:
### **Ejercicios**

1. Lee y visualiza el vídeo `tra╠üfico01.mp4`.
2. Promedia las imágenes del vídeo para obtener un fondo sin coches.
3. Resta el fondo a cada imagen del vídeo.
4. Umbraliza la imagen resultante para quedarte únicamente con las zonas donde hay diferencias significativas en el valor de los píxeles.
4. Detecta los blobs que correspondan a coches, ten en cuenta que los coches son más grandes que el ruido.

In [1]:
#apartado 1.1
import cv2

video = 'tra╠üfico01.mp4'  #ruta del video
cap = cv2.VideoCapture(video)  #lee video

#lee todos los frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imshow('Video', frame)
    if cv2.waitKey(20) & 0xFF == ord('q'): #muestra vídeo entero frame a frame
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
#apartado 1.2
import cv2
import numpy as np

video = 'tra╠üfico01.mp4'
cap = cv2.VideoCapture(video)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  #nº total frames
ret, frame = cap.read()  #lee primer frame, saber tamaño
if not ret or frame is None or frame_count == 0:
    print("Error, no se puede leer el vídeo.")
    cap.release()
    exit()

avg_frame = np.zeros_like(frame, dtype=np.float32)  #crea imagen para acumular suma, tipo flotante para no problema
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  #volvemos primer frame

valid_frames = 0
for i in range(frame_count):
    ret, frame = cap.read()
    if not ret or frame is None:
        continue
    avg_frame += frame.astype(np.float32)
    valid_frames += 1

if valid_frames == 0:
    print("Error, no se puede leer ningún frame.")
    cap.release()
    exit()

avg_frame /= valid_frames
avg_frame = np.nan_to_num(avg_frame)
background = avg_frame.astype(np.uint8)

cv2.imshow('background promedia', background)
cv2.imwrite('background.png', background)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
#apartado 1.3
import cv2
import numpy as np

video = 'tra╠üfico01.mp4'
background = cv2.imread('background.png')  # Imagen del fondo promedio

if background is None:
    print("Error: No se pudo cargar 'background.png'.")
    exit()

cap = cv2.VideoCapture(video)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break
    # Resta el fondo al frame actual
    diff = cv2.absdiff(frame, background)
    # Muestra el resultado
    cv2.imshow('Resta del fondo', diff)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [4]:
#apartado 1.4
import cv2
import numpy as np

video = 'tra╠üfico01.mp4'
background = cv2.imread('background.png')

if background is None:
    print("Error: No se pudo cargar 'background.png'.")
    exit()

cap = cv2.VideoCapture(video)

T = 50  # Ajusta este valor para definir qué es una diferencia significativa

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break
    diff = cv2.absdiff(frame, background)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, diff_bin = cv2.threshold(diff_gray, T, 255, cv2.THRESH_BINARY)
    cv2.imshow('Diferencias significativas', diff_bin)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# VEHICULOS DETECTADOS 1.5 (mínimo 5)

import cv2
import numpy as np

video = 'tra╠üfico01.mp4'
background = cv2.imread('background.png')

if background is None:
    print("Error: No se pudo cargar 'background.png'.")
    exit()

cap = cv2.VideoCapture(video)

T = 50

# Lee el primer frame para obtener tamaño
ret, frame = cap.read()
if not ret or frame is None:
    print("Error: No se pudo leer el primer frame.")
    exit()

height, width = frame.shape[:2]
rect_width = 60
rect_height = 100
top_left_x = (width - rect_width) - 1350
top_left_y = (height - rect_height) // 2

area_pts = np.array([
    [top_left_x, top_left_y],
    [top_left_x + rect_width, top_left_y],
    [top_left_x + rect_width, top_left_y + rect_height],
    [top_left_x, top_left_y + rect_height]
])

class Coche:
    def __init__(self, x, y, w, h, area, id):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.area = area
        self.id = id
        self.frames_seen = 1
        self.updated = True

    def centroide(self):
        # Retorna el centro del bounding box
        return (self.x + self.w // 2, self.y + self.h // 2)

coches_unicos = []
proximo_id = 1  # ID incremental de coche

# Regresamos al primer frame del video
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
DISTANCIA_MAX = 50  # Max distancia en píxeles para considerar "el mismo coche"

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break

    diff = cv2.absdiff(frame, background)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, diff_bin = cv2.threshold(diff_gray, T, 255, cv2.THRESH_BINARY)
    mask = np.zeros_like(diff_bin)
    cv2.fillPoly(mask, [area_pts], 255)
    diff_bin_carril = cv2.bitwise_and(diff_bin, mask)
    contours, _ = cv2.findContours(diff_bin_carril, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detectados = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 500:
            x, y, w, h = cv2.boundingRect(contour)
            detectados.append((x, y, w, h, area))

    # Marcar todos los coches como NO actualizados
    for coche in coches_unicos:
        coche.updated = False

    # Para cada detección, buscar el coche anterior más cercano (tracking simple por centroide)
    for x, y, w, h, area in detectados:
        c_actual = (x + w // 2, y + h // 2)
        min_dist = float('inf')
        coche_match = None
        for coche in coches_unicos:
            c_anterior = coche.centroide()
            dist = np.hypot(c_actual[0] - c_anterior[0], c_actual[1] - c_anterior[1])
            if dist < min_dist and dist < DISTANCIA_MAX:
                min_dist = dist
                coche_match = coche
        if coche_match is not None:
            # Actualiza el coche existente
            coche_match.x = x
            coche_match.y = y
            coche_match.w = w
            coche_match.h = h
            coche_match.area = area
            coche_match.frames_seen += 1
            coche_match.updated = True
            coche_id = coche_match.id
        else:
            # Coche NUEVO
            nuevo_coche = Coche(x, y, w, h, area, proximo_id)
            coches_unicos.append(nuevo_coche)
            proximo_id += 1
            coche_id = nuevo_coche.id
        # Dibuja el bounding box con el ID
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f'ID:{coche_id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Puedes eliminar coches que llevan mucho sin aparecer, aquí queremos mantener el historial

    cv2.putText(frame, f'Contador coches: {len(coches_unicos)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.polylines(frame, [area_pts], isClosed=True, color=(255, 0, 0), thickness=2)
    cv2.imshow('Video con detección de coches únicos', frame)
    cv2.imshow('Diferencias significativas', diff_bin)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f'Coches únicos detectados en el vídeo: {len(coches_unicos)}')


# Liberar recursos
cap.release()
cv2.destroyAllWindows()

Coches únicos detectados en el vídeo: 5


In [ ]:
# resto de ejercicios
import cv2
import numpy as np

video = 'tra╠üfico01.mp4'
background = cv2.imread('background.png')

if background is None:
    print("Error: No se pudo cargar 'background.png'.")
    exit()

cap = cv2.VideoCapture(video)

T = 50                   # umbral para binarizar la diferencia
AREA_MINIMA = 100        # área mínima de contorno para considerar detección
AREA_MAX = 15000         # área máxima de contorno para considerar detección
DISTANCIA_MAX = 78       # distancia máxima para considerar "el mismo coche"
MAX_FRAMES_PERDIDO = 10  # nº máximo de frames sin ver un coche antes de borrarlo

# Lee el primer frame para obtener tamaño
ret, frame = cap.read()
if not ret or frame is None:
    print("Error: No se pudo leer el primer frame.")
    exit()

height, width = frame.shape[:2]

# configuración para cada vía 
widths  = [60, 70, 70, 200, 200, 100]     # ancho de las vías
heights = [35, 100, 100, 100, 90, 70]     # alturas 
gaps    = [70, 50, 350, -20, -50]         # gaps entre vías (distancia de via a via)
y_offsets = [-250, 70, 0, 150, 30, -135]  # offsets verticales por vía (ajusta para poner más arriba o más abajo)

USE_EXPLICIT_VIAS = False
vias_explicit = [] 

# via que esta más a la izquierda
EXTRA_LEFT = 250  # píxeles para mover la primera vía aún más a la izquierda
first_rect_width = widths[0] if len(widths) > 0 else 60
base_top_left_x = (width - first_rect_width) - 1350 - EXTRA_LEFT


# Construimos varias áreas con tamaños, gaps y offsets verticales distintos
area_pts_list = []

if USE_EXPLICIT_VIAS and len(vias_explicit) > 0:
    for (x, y, w, h) in vias_explicit:
        x0 = max(0, min(int(x), width-1))
        y0 = max(0, min(int(y), height-1))
        x1 = min(width-1, x0 + int(w))
        y1 = min(height-1, y0 + int(h))
        area_pts = np.array([[x0,y0],[x1,y0],[x1,y1],[x0,y1]])
        area_pts_list.append(area_pts)
else:
    num_vias = len(widths)  # longitud determina NUM_VIAS

    # Asegurar que heights, gaps y y_offsets tengan la longitud necesaria
    if len(heights) < num_vias:
        heights = heights + [heights[-1]] * (num_vias - len(heights))
    if len(gaps) < max(0, num_vias-1):
        gaps = gaps + [0] * (max(0, num_vias-1) - len(gaps))
    if len(y_offsets) < num_vias:
        y_offsets = y_offsets + [0] * (num_vias - len(y_offsets))

    tlx = base_top_left_x
    for i in range(num_vias):
        wv = int(widths[i])
        hv = int(heights[i])
        # centrar verticalmente cada vía según su altura y aplicar offset vertical específico
        tly = (height - hv) // 2 + int(y_offsets[i])
        # Clamp para no salir del frame
        x0 = max(0, min(tlx, width-1))
        y0 = max(0, min(tly, height-1))
        x1 = min(width-1, x0 + wv)
        y1 = min(height-1, y0 + hv)
        area_pts = np.array([
            [x0, y0],
            [x1, y0],
            [x1, y1],
            [x0, y1]
        ])
        area_pts_list.append(area_pts)
        # mover X acumulativo: ancho de esta vía + gap (cuando existe)
        gap = gaps[i] if i < len(gaps) else 0
        tlx = tlx + wv + gap

# inicialización de los contadores por vía
counters_vias = [0 for _ in range(len(area_pts_list))]

class Coche:
    def __init__(self, x, y, w, h, area, id, via_index=None):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.area = area
        self.id = id
        self.frames_seen = 1
        self.updated = True
        self.via_index = via_index  # vía en la que fue detectado inicialmente
        self.frames_perdido = 0     # nº de frames seguidos sin ser actualizado

    def centroide(self):
        # Retorna el centro del bounding box
        return (self.x + self.w // 2, self.y + self.h // 2)

coches_unicos = []
proximo_id = 1        # ID incremental de coche
total_coches = 0      # número total de coches distintos creados

# volvemos al primer frame del video
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        break

    diff = cv2.absdiff(frame, background)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, diff_bin = cv2.threshold(diff_gray, T, 255, cv2.THRESH_BINARY)

    
    detectados = []  # guardamos (x, y, w, h, area, via_index) por detección

    # Para cada vía, aplicamos su máscara y extraemos contornos, para evitar duplicados cuando hay solapamiento entre vías, se guardarán los centros ya asignados
    centros_asignados = set()
    for idx, area_pts in enumerate(area_pts_list):
        mask = np.zeros_like(diff_bin)
        cv2.fillPoly(mask, [area_pts], 255)
        diff_bin_carril = cv2.bitwise_and(diff_bin, mask)
        contours, _ = cv2.findContours(diff_bin_carril, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if AREA_MINIMA <= area <= AREA_MAX:
                x, y, wv, hv = cv2.boundingRect(contour)
                cX = x + wv//2
                cY = y + hv//2
                centro_key = (int(cX), int(cY))
                # Si ya asignamos una detección con el mismo centro, la saltamos
                if centro_key in centros_asignados:
                    continue
                centros_asignados.add(centro_key)
                detectados.append((x, y, wv, hv, area, idx))

    # Marcar todos los coches como no actualizados
    for coche in coches_unicos:
        coche.updated = False

    # Para cada detección, buscar el coche anterior más cercano
    for x, y, wv, hv, area, via_idx in detectados:
        c_actual = (x + wv // 2, y + hv // 2)
        min_dist = float('inf')
        coche_match = None
        for coche in coches_unicos:
            c_anterior = coche.centroide()
            dist = np.hypot(c_actual[0] - c_anterior[0], c_actual[1] - c_anterior[1])
            if dist < min_dist and dist < DISTANCIA_MAX:
                min_dist = dist
                coche_match = coche

        if coche_match is not None:
            # se actualiza el coche existente
            coche_match.x = x
            coche_match.y = y
            coche_match.w = wv
            coche_match.h = hv
            coche_match.area = area
            coche_match.frames_seen += 1
            coche_match.updated = True
            coche_match.frames_perdido = 0
            coche_id = coche_match.id
            # Si aún no tiene vía asignada, y la detección está en una vía, se le asigna
            if coche_match.via_index is None and via_idx is not None:
                coche_match.via_index = via_idx
                counters_vias[via_idx] += 1
        else:
            # nuevo coche
            nuevo_coche = Coche(x, y, wv, hv, area, proximo_id, via_index=via_idx)
            coches_unicos.append(nuevo_coche)
            if via_idx is not None:
                counters_vias[via_idx] += 1
            proximo_id += 1
            total_coches += 1        # contamos el coche nuevo como uno distinto
            coche_id = nuevo_coche.id

        # se dibuja el bounding box con el ID
        cv2.rectangle(frame, (x, y), (x + wv, y + hv), (0, 255, 0), 2)
        cv2.putText(frame, f'ID:{coche_id}', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # eliminamos lo coches que ya no aparecen
    coches_filtrados = []
    for coche in coches_unicos:
        if coche.updated:
            coches_filtrados.append(coche)
        else:
            coche.frames_perdido += 1
            if coche.frames_perdido <= MAX_FRAMES_PERDIDO:
                coches_filtrados.append(coche)
            # si lo hemos perdido demasiados frames, lo quitamos de la lista
    coches_unicos = coches_filtrados

    # dibujamos todas las vías y sus contadores
    for idx, area_pts in enumerate(area_pts_list):
        color = (255, 0, 0)
        cv2.polylines(frame, [area_pts], isClosed=True, color=color, thickness=2)
        px = int(area_pts[0][0]) + 5
        py = int(area_pts[0][1]) - 10
        cv2.putText(frame, f'Via {idx+1}: {counters_vias[idx]}', (px, py),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)

    # usamos total_coches, es decir, coches distintos en lugar de len(coches_unicos)
    cv2.putText(frame, f'Contador coches: {total_coches}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow('Video con detección de coches únicos', frame)
    cv2.imshow('Diferencias significativas', diff_bin)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f'Coches únicos detectados en el vídeo: {total_coches}')
for idx, cnt in enumerate(counters_vias):
    print(f'Via {idx+1} - Coches detectados: {cnt}')

Coches únicos detectados en el vídeo: 77
Via 1 - Coches detectados: 8
Via 2 - Coches detectados: 5
Via 3 - Coches detectados: 5
Via 4 - Coches detectados: 27
Via 5 - Coches detectados: 18
Via 6 - Coches detectados: 14
